In [1]:
%load_ext autoreload
from ddls.plotting.plotting import plot_line, plot_bar, plot_hist

from collections import defaultdict
from sqlitedict import SqliteDict
import pprint
import glob
import time
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
%autoreload

# load files
base_folder = '/scratch/datasets/ddls/sims/'
base_name = 'cluster'
ids = [22, 24, 26]


steps_logs_dict = defaultdict(lambda : defaultdict(list))
sim_logs_dict = defaultdict(lambda : defaultdict(list))
start_time = time.time()
for i in ids:
    agent = base_name + f'_{i}'
    paths = [reset_folder for reset_folder in glob.glob(base_folder + f'/{base_name}/{agent}/*/')]
    print(f'i: {i} | agent: {agent} | paths: {paths}')
    
    for path in paths:
        with SqliteDict(path + '/steps_log.sqlite') as log:
            for key, val in log.items():
                steps_logs_dict[agent][key].extend(val)
            log.close()

        with SqliteDict(path + '/sim_log.sqlite') as log:
            for key, val in log.items():
                sim_logs_dict[agent][key].extend(val)
            log.close()

i: 22 | agent: cluster_22 | paths: ['/scratch/datasets/ddls/sims//cluster/cluster_22/reset_2/', '/scratch/datasets/ddls/sims//cluster/cluster_22/reset_1/', '/scratch/datasets/ddls/sims//cluster/cluster_22/reset_3/']
i: 24 | agent: cluster_24 | paths: ['/scratch/datasets/ddls/sims//cluster/cluster_24/reset_2/', '/scratch/datasets/ddls/sims//cluster/cluster_24/reset_1/', '/scratch/datasets/ddls/sims//cluster/cluster_24/reset_3/']
i: 26 | agent: cluster_26 | paths: ['/scratch/datasets/ddls/sims//cluster/cluster_26/reset_2/', '/scratch/datasets/ddls/sims//cluster/cluster_26/reset_1/', '/scratch/datasets/ddls/sims//cluster/cluster_26/reset_3/']


In [ ]:
# augment logs dict as desired

# def augment_steps_logs_dict(steps_logs_dict):
#     for agent in steps_logs_dict.keys():
        
# #         wallclock time
# #         steps_logs_dict[agent]['wallclock_time'] = [steps_logs_dict[agent]['step_end_time'][0]]
# #         for step_end_time in steps_logs_dict[agent]['step_end_time'][1:]:
# #             steps_logs_dict[agent]['wallclock_time'].append(steps_logs_dict[agent]['wallclock_time'][-1] + step_end_time)
#     steps_logs_dict = []
    
#     return steps_logs_dict
            
# steps_logs_dict = augment_steps_logs_dict(steps_logs_dict)
        
print(f'\nsteps_logs_dict: {steps_logs_dict}\n')
print(f'\nsim_logs_dict: {sim_logs_dict}\n')
print(f'\nAll data loaded in {time.time() - start_time:.3f} s.')

# Steps logs plots

In [ ]:
%autoreload

# plot config
# x = 'step_counter'
x = 'step_end_time'
scaling_factor = 1
metrics_to_skip = {x}

# make plots
metrics = steps_logs_dict[agent].keys()
metrics_to_plot = sorted([metric for metric in metrics if metric not in metrics_to_skip])
print(f'Metrics to plot: {metrics_to_plot}\nMetrics to skip: {metrics_to_skip}\n')
for metric in metrics_to_plot:
    print(f'Plotting metric {metric}')
    plot_dict = defaultdict(list)
    for _agent in steps_logs_dict.keys():
        plot_dict['Agent'].extend([_agent for _ in range(len(steps_logs_dict[_agent][metric]))])
        plot_dict[x].extend(steps_logs_dict[_agent][x])
        plot_dict[metric].extend(steps_logs_dict[_agent][metric])
    fig = plot_line(pd.DataFrame(plot_dict), 
                    x=x, 
                    y=metric, 
                    hue='Agent', 
                    xlabel=x, 
                    ylabel=metric, 
                    err_style='band', # 'band' 'bars'
                    ci=68, # 95 68
                    scaling_factor=scaling_factor,
                    show_fig=True)

# Sim logs plots

In [ ]:
%autoreload

# plot config
scaling_factor = 1
metrics_to_skip = {}
def p99(a):
    return np.percentile(a, 99)
estimators = {'mean': np.mean,
              'p99': p99,
              'iqr': st.iqr,
              'gmean': st.gmean}


# make plots
metrics = sim_logs_dict[agent].keys()
metrics_to_plot = sorted([metric for metric in metrics if metric not in metrics_to_skip])
print(f'Metrics to plot: {metrics_to_plot}\nMetrics to skip: {metrics_to_skip}\n')
for metric in metrics_to_plot:
    print(f'Plotting metric {metric}')
    plot_dict = defaultdict(list)
    for _agent in sim_logs_dict.keys():
        plot_dict['Agent'].extend([_agent for _ in range(len(sim_logs_dict[_agent][metric]))])
        plot_dict[metric].extend(sim_logs_dict[_agent][metric])
        
    df = pd.DataFrame(plot_dict)
        
    # hist
    fig = plot_hist(df,
                    x=metric,
                    hue='Agent',
                    xlabel=metric,
                    element='bars',
                    fill=True,
                    cumulative=False,
                    stat='count',
                    multiple='layer',
                    scaling_factor=scaling_factor,
                    show_fig=True)
    
    # cdf
    fig = plot_hist(df,
                    x=metric,
                    hue='Agent',
                    xlabel=metric,
                    element='step',
                    fill=False,
                    cumulative=True,
                    stat='density',
                    common_norm=False,
                    scaling_factor=scaling_factor,
                    show_fig=True)
        
    # bar chart
    for estimator_name, estimator in estimators.items():
        fig = plot_bar(df, 
                        x='Agent', 
                        y=metric, 
                        xlabel='Agent', 
                        ylabel=metric, 
                        estimator=estimator,
                        title=estimator_name,
                        scaling_factor=scaling_factor,
                        show_fig=True)